In [ ]:
# Checking python version
!python --version

Python 3.10.12


In [ ]:
# Download and install Vieira
# The default python version is for 3.10, you may change the link according to your python versions.
%%capture
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira-0.2.2-cp310-cp310-manylinux_2_31_x86_64.whl
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira_ext-0.2.2-py3-none-any.whl
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira_gpu-0.0.1-py3-none-any.whl
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira_gpt-0.0.1-py3-none-any.whl
!pip install vieira-0.2.2-cp310-cp310-manylinux_2_31_x86_64.whl
!pip install vieira_ext-0.2.2-py3-none-any.whl
!pip install vieira_gpu-0.0.1-py3-none-any.whl
!pip install vieira_gpt-0.0.1-py3-none-any.whl

In [ ]:
# Import Vieira and related plugins
import vieira
import vieira_ext

# Vieira with Embeddings

In [ ]:
# Add your OpenAI API key if you want to run the example
import os
os.environ['OPENAI_API_KEY'] =  "YOUR-OPENAI-API-KEY-HERE"

In [ ]:
# Configure Vieira plugins
import argparse
plugins = vieira_ext.PluginRegistry()

parser = argparse.ArgumentParser()
plugins.setup_argument_parser(parser)
known_args, unknown_args = parser.parse_known_args()
plugins.configure(known_args, unknown_args)

## Import dataset

In [ ]:
!wget https://raw.githubusercontent.com/vieira-artifact/vieira-artifact-aaai24/main/res/product_search_examples.json

--2023-11-06 01:09:42--  https://raw.githubusercontent.com/vieira-artifact/vieira-artifact-aaai24/main/res/product_search_examples.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68821 (67K) [text/plain]
Saving to: ‘product_search_examples.json’

product_search_exam 100%[===================>]  67.21K  --.-KB/s    in 0.007s  

2023-11-06 01:09:42 (9.30 MB/s) - ‘product_search_examples.json’ saved [68821/68821]



In [ ]:
import json
dataset = json.load(open('product_search_examples.json', 'r'))

In [ ]:
# Amazon Product Search

PROGRAM = """
type question(q: String)

type product(id: i32, p: String)
type gt_relevance(id: i32, l: String)

@gpt_encoder
type $embed_text(String) -> Tensor

@gpt(
  header="Please return only a json list of the corresponding {{pos}} and {{neg}} values.",
  prompt="Given a query {{q}}, it is asking for {{pos}} but not {{neg}}",
  examples=[
    ("zipper tape without pulls", "zipper tape", "pulls"),
    ("!awnmower tires without rims", "lawnowner tires", "rims"),
    ("$1 items for men", "$1 items for men", "EMPTY")
  ],
  model="gpt-4",
  debug=false,
)
type extract_pos_neg(bound q: String, pos: String, neg: String)

rel pos_attr(p) = question(q) and extract_pos_neg(q, p, _)
rel neg_attr(n) = question(q) and extract_pos_neg(q, _, n)

rel match_pos(pid) = product(pid, ptitle) and pos_attr(p) and soft_eq<Tensor>($embed_text(p), $embed_text(ptitle))
rel match_neg(pid) = product(pid, _) and neg_attr("EMPTY")
rel match_neg(pid) = product(pid, ptitle) and neg_attr(n) and soft_neq<Tensor>($embed_text(n), $embed_text(ptitle))

rel match(pid) = match_pos(pid) and match_neg(pid)
query match

rel check_all(pid, p, gt) = match(pid) and gt_relevance(pid, gt) and product(pid, p)
query check_all
"""

for idx, example in enumerate(dataset):
  ctx = vieira.Context(provenance='topkproofs')
  plugins.load_into_ctx(ctx)
  ctx.add_program(PROGRAM)
  ctx.set_non_probabilistic(['question', 'product', 'gt_relevance'])

  query = example['query']
  ctx.add_facts('question', [(query,)])
  for product in example['products']:
    ctx.add_facts('product',      [(product['product_id'], product['product_name'],)])
    ctx.add_facts('gt_relevance', [(product['product_id'], product['product_relevance'],)])

  ctx.run()

  result = list(ctx.relation('check_all'))
  result.sort(key=lambda tup: tup[0], reverse=True)
  out_str = f"Example {idx}\n query: {query}\n\n"
  for _, (_, product, rel) in result:
    out_str += rel + product + '\n'

  print(out_str + '\n\n')

Example 0:	 query: .5 inch suction cups without hooks
Aoyoho Diameter 20MM Mini Clear Plastic Suction Cups Without Hooks Pack of 120E
Hyamass 100pcs Mini Clear Plastic Suction Cups Without Hooks (Diameter 30mm)S
Outus Small Clear Plastic Suction Cup Without Hooks -20 mm -20 PacksS
Antrader Diameter 30MM Transparent Plastic Suction Cups Anti-Collision Sucker Hanger Pads Without Hooks 8-PackS
Whaline 45mm Large Suction Cup PVC Plastic Sucker Pads Without Hooks, 20 Packs ClearS
Whaline 30 Packs Suction Cup Plastic Sucker Pads Without Hooks, Clear, 3 Size, 45 mm, 30 mm, 20 mmS
BEADNOVA Suction Cups for Glass 2.5 Inches Suction Hooks Window Suction Cups with Hooks for Home Kitchen Bathroom Wall (12 Packs)S
Weoxpr 90 Pieces Clear Plastic Suction Cup Without Hooks, 3 Size Sucker Pads Hanger Pads for Home Glass Table Desk, 20 mm, 30 mm, 45 mmS
Shapenty 20mm Diameter Small Clear PVC Plastic Suction Cups Home Desk Glass Sucker Hanger Pads Without Hooks, 20PiecesS
Pawfly 20 Pack Clear Suction Cup